In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [31]:
### Load the dataset

data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [32]:
# as we can see, the row number and customer id dont really seem like
# important features. Using the remaining features, we are going to predict
# whether the customer has exited the bank or not


# we will be dropping irrelavant features like rownumber and customerid
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1) # column wise
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [33]:
# we have a column called as geography.
data['Geography'].unique()

# geography colu

array(['France', 'Spain', 'Germany'], dtype=object)

In [34]:
# let us encode the gender column using Label Encoder

LabelEncoder_gender = LabelEncoder()
data['Gender'] = LabelEncoder_gender.fit_transform(data['Gender'])

In [35]:
## Onehot encode 'Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [36]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [37]:
# now we will combine the one hot encoded columns along with the original data

data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [11]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [38]:
# saving the encoders

with open('LabelEncoder_gender.pkl','wb') as file:
    pickle.dump(LabelEncoder_gender,file)

with open('OneHotEncoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [39]:
# Divide the data set into independent and dependent features

data.head() # exited is the column that we will be using as the dependent/output variable
# all the remaining columns will be the independent features

X = data.drop('Exited',axis=1)
y = data['Exited']

# split the data in training and testing sets

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Scale down the features

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [40]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### ANN Implementation

In [41]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [42]:
# Build our ANN model

model = Sequential([
    Dense(64,activation = 'relu',input_shape=(X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32,activation='relu'), ## Hidden layer 2
    Dense(1,activation='sigmoid') ## output layer
]

)

In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01) # use this adam and you can provide your own learing rate

In [45]:
# In order to do forward and backward propogation, we need to compile this model.

model.compile(optimizer =opt,loss="binary_crossentropy",metrics=['accuracy'])

# is this the only way to put any values?? no, there are other ways as well

In [46]:
# we did not start the training yet

## lets setup the TensorBoard so that we can capture and visualize the training

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [47]:
# Set up early stopping

# After a certain number of epochs, the lsos is not really varying by a huge amount,
# we can go ahead and stop. We dont really need to run all the eopchs as that would just be 
# a waste of time and pointless.

early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True) 
# be patient for atleast 5 epochs before stopping

In [48]:
# Training the model

history = model.fit(
    X_train,y_train,validation_data = (X_test,y_test),epochs=100,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 783us/step - loss: 0.4069 - accuracy: 0.8315 - val_loss: 0.3485 - val_accuracy: 0.8590
Epoch 2/100
250/250 [==============================] - 0s 523us/step - loss: 0.3533 - accuracy: 0.8537 - val_loss: 0.3496 - val_accuracy: 0.8575
Epoch 3/100
250/250 [==============================] - 0s 520us/step - loss: 0.3466 - accuracy: 0.8585 - val_loss: 0.3730 - val_accuracy: 0.8570
Epoch 4/100
250/250 [==============================] - 0s 528us/step - loss: 0.3430 - accuracy: 0.8564 - val_loss: 0.3400 - val_accuracy: 0.8660
Epoch 5/100
250/250 [==============================] - 0s 517us/step - loss: 0.3363 - accuracy: 0.8622 - val_loss: 0.3629 - val_accuracy: 0.8640
Epoch 6/100
250/250 [==============================] - 0s 512us/step - loss: 0.3384 - accuracy: 0.8602 - val_loss: 0.3402 - val_accuracy: 0.8620
Epoch 7/100
250/250 [==============================] - 0s 519us/step - loss: 0.3349 - accuracy: 0.8610 - val_loss: 0.3362 - val_ac

In [49]:
model.save('model.h5')

/Users/srikarasai/Desktop/Projects/Ann_project/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [81]:
## load Tensorboard Extension

%load_ext tensorboard

In [83]:
%tensorboard --logdir logs/fit

## we need to load the pickle files for testing

In [84]:
OneHotEncoder_geography.

,categories,'auto'
,drop,None
,sparse_output,False
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'
